
可选: 数据并行
==========================
**作者**: `Sung Kim <https://github.com/hunkim>`_ 和 `Jenny Kang <https://github.com/jennykang>`_

在这个教程中, 我们将会学习如何在多个GPU上使用 ``DataParallel`` .

在 PyTorch 中使用 GPU 是一件很容易的事情.你可以像下面这样轻松的将一个模型分配到一个 GPU 上.


`model.gpu()`

随后, 你可以将你的所有张量拷贝到上面的GPU:


`mytensor = my_tensor.gpu()`

此处请注意: 如果只是调用 ``mytensor.gpu()`` 是不会将张量拷贝到 GPU 的.你需要将它赋给一个
新的张量, 这个张量就能在 GPU 上使用了.

在多个 GPU 上运行前向、反向传播是一件很自然的事情, 然而,  PyTorch 默认情况下只会用到一个GPU, 
可以通过使用 ``DataParallel`` 使你的模型并行运行, 在多个GPU上运行这些操作也将变得非常简单:

`model = nn.DataParallel(model)`

这是教程的核心内容, 我们将在随后进行详细讲解



导入和参数
----------------------

导入PyTorch模块和参数定义




In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

# 参数和数据加载
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

伪数据集
-------------

只需要实现 getitem 就可以轻松的生成一个（随机）伪数据集, 如下代码所示: 




In [2]:
class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset=RandomDataset(input_size, 100),
                         batch_size=batch_size, shuffle=True)

简单模型
------------

在下面的示例中, 我们的模型只需要一个输入并且完成一个线性操作, 最后得
到一个输出.当然, 你可以在任意模型 (CNN,RNN,Capsule Net等) 运用  ``DataParallel``  

我们在模型中设置了打印指令来监控输入和输出的张量大小, 请注意批数据次序为0时的输出.




In [3]:
class Model(nn.Module):
    # Our model

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)
        print("  In Model: input size", input.size(), 
              "output size", output.size())

        return output

创建模型和 DataParallel
-----------------------------

这是本教程的核心部分. 首先, 我们需要生成一个模型的实例并且检测我们是否拥有多个
GPU.如果有多个GPU , 我们可以使用 ``nn.DataParallel`` 来包装我们的模型, 然后我们
就可以将我们的模型通过 ``model.gpu()`` 施加于这些GPU上.




In [6]:
torch.cuda.device_count()

7L

In [7]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  model = nn.DataParallel(model)

if torch.cuda.is_available():
   model.cuda()

("Let's use", 7L, 'GPUs!')


运行模型
-------------

现在我们可以看到输入和输出张量的大小了.




In [18]:
for data in rand_loader:
    if torch.cuda.is_available():
        input_var = Variable(data.cuda())
    else:
        input_var = Variable(data)
    output = model(input_var)
    print('00000')
    print("Outside: input size", input_var.size(), "output_size", output.size())
    print('11111')

('  In Model: input size', torch.Size([5, 5]), 'output size', torch.Size([5, 2]))
('  In Model: input size', torch.Size([5, 5]), 'output size', torch.Size([5, 2]))('  In Model: input size', torch.Size([5, 5]), 'output size', torch.Size([5, 2]))

('  In Model: input size', torch.Size([5, 5]), 'output size', torch.Size([5, 2]))
('  In Model: input size', torch.Size([5, 5]), 'output size', torch.Size([5, 2]))
('  In Model: input size', torch.Size([5, 5]), 'output size', torch.Size([5, 2]))
00000
('Outside: input size', torch.Size([30, 5]), 'output_size', torch.Size([30, 2]))
11111
('  In Model: input size', torch.Size([5, 5]), 'output size', torch.Size([5, 2]))
('  In Model: input size', torch.Size([5, 5]), 'output size', torch.Size([5, 2]))
('  In Model: input size', torch.Size([5, 5]), 'output size', torch.Size([5, 2]))
('  In Model: input size', torch.Size([5, 5]), 'output size', torch.Size([5, 2]))
('  In Model: input size', torch.Size([5, 5]), 'output size', torch.Size([5, 2]))
('  I

结果
-------

当我们将输入设置为30批, 模型也产生了30批的输出.但是当我们使用多个GPU, 然后你
会得到类似下面这样的输出.

### 2 GPUs


如果有2个GPU, 我们将会看到这样的结果:


    # on 2 GPUs
    Let's use 2 GPUs!
        In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
        In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
        In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
        In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2])
        In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2])
    Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])

### 3 GPUs


如果有3个GPU, 我们将会看到这样的结果:

    Let's use 3 GPUs!
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
        In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])

### 8 GPUs


如果有8个GPU, 我们将会看到这样的结果:

    Let's use 8 GPUs!
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
        In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])




总结
-------

DataParallel 自动地将数据分割并且将任务送入多个GPU上的多个模型中进行处理.
在每个模型完成任务后,  DataParallel 采集和合并所有结果, 并将最后的结果呈现给你.

想了解更多信息, 请点击:
http://pytorch.org/tutorials/beginner/former\_torchies/parallelism\_tutorial.html.




# pytorch 多GPU训练
转载：[pytorch 多GPU训练](https://blog.csdn.net/jacke121/article/details/80605205)

pytorch多GPU最终还是没搞通，可用的部分是前向计算，back propagation会出错，当时运行通过，也不太确定是如何通过了的。目前是这样，有机会再来补充

pytorch支持多GPU训练，官方文档（pytorch 0.30）给了一些说明：pytorch数据并行，但遗憾的是给出的说明并不详细。不过说的还是蛮清楚的，建议使用`DataParallel`。

pytorch使用多GPU训练的时候要考虑的主要的不过是前向计算和后向计算两个部分。
## 前向计算：

In [ ]:
net = Net() #Net是自定义的一个网络结构类  
device_ids = [2, 4, 5]  
cudnn.benchmark = True  
net = net.cuda(device_ids[0])  
net = nn.DataParallel(net, device_ids=device_ids) #使用dataParallel重新包装一下  

`Dataparallel`会把输入数据以某个轴（默认是 0 轴）将数据分开成平均几份，所以要保证batch size每一个批中的图片数目是相对于用到的GPU可除的。等调用`net`结束后，将再次把分开运算得到的数据自动组合在一起。
## 后向计算：
神经网络还需要后向计算以更新梯度，因为`grad_fn`在`dataparallel`中，所以也要将更新梯度的方法放入到`dataparallel`中去。

In [ ]:
lr = 1e-2  
momentum = 0.9  
weight_decay = 1e-3  
param = get_param(net, lr)  
optimizer = optim.SGD(param, momentum=momentum, weight_decay=weight_decay) #准备pytorch中的随机梯度下降方法  
loss = nn.MSEloss()   
optimizer = nn.DataParallel(optimizer, device_ids=device_ids) #将optimizer放入dataparallel中。  

## DataParallel的使用：

In [ ]:
img = Variable(img, requires_grad=True).cuda(device_ids[0])  # 输入图片数据  
gt = Variable(gt_heatmap).cuda(device_ids[0]) #ground truth  
  
predicted = net(img) # net是DataParallel对象,img 作为输入会将分为3份（bachsize/3），等3个并行计算结束后再以该轴组合再一起，predicted和img的shape是一样的。  
l = loss(gt, predicted)   
  
# compute gradient and do SGD step  
optimizer.zero_grad()   
l.backward() #在这儿使用optimizer的相应的对象。  
optimizer.module.step() #因为它在DataParallel里面，所以要先变成普通的nn.SGD对象，然后才能调用该类的梯度更新方法。类似的，还有其他的一些需要注意的地方，看下面：  

## 相应的学习率更新的方法：


In [ ]:
for param_lr in optimizer.module.param_groups: #同样是要加module  
    param_lr['lr'] /= 2  

## 加载保存的网络参数：

加载保存的网络参数时也要注意，因为所有的保存的参数对应的关键字都加了`module`。可以像下面这样使用序号的方式重新加载所保存的网络参数。

In [ ]:
model_dict = net.state_dict()  
vgg_19_key = list(vgg_19.keys())  
model_key = list(model_dict.keys())  
from collections import OrderedDict  
vgg_dict = OrderedDict()  
for i in range(param_num):  
    vgg_dict[model_key[i]] = vgg_19[vgg_19_key[i]]  
model_dict.update(vgg_dict)  

也可以简单的去掉`OrderedDict`关键字多出的`module`，像这样：

In [ ]:
for item, value in saved_state.items():  
    name = '.'.join(item.split('.')[1:])  
    trans_param[name] = value  

像这样就可以实现pytorch多GPU训练网络了。